In [1]:
from absl import logging
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import fnmatch
from collections import defaultdict
import tqdm
import hashlib
import tensorflow as tf

In [4]:
cur_dir = os.getcwd()
dictFiles = defaultdict(list)
# Get all the filepaths for json and respective image files
for dirpath, dirname, filenames in os.walk('./data/Audi_dataset'):
    for file in filenames:
        if fnmatch.fnmatch(file, '*_label3D_*.json'):
            key = file.split('_')[3].split('.')[0].lstrip('0')
            filepath = dirpath + '/' + file
            dictFiles[key].append(filepath.lstrip('.'))
        if fnmatch.fnmatch(file, '*_camera_*.png'):
            key = file.split('_')[3].split('.')[0].lstrip('0')
            filepath = dirpath + '/' + file
            dictFiles[key].append(filepath.lstrip('.'))
# DataFrame with all the filepaths for json and respective image files
dataset = pd.DataFrame([(k, v[0],v[1]) for k, v in dictFiles.items()], columns = ['id','jsonFile','imageFile'])

dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11754 entries, 0 to 11753
Data columns (total 3 columns):
id           11754 non-null object
jsonFile     11754 non-null object
imageFile    11754 non-null object
dtypes: object(3)
memory usage: 275.6+ KB


,id,jsonFile,imageFile
0,122788,/data/Audi_dataset/20181204_135952/label3D/cam...,/data/Audi_dataset/20181204_135952/camera/cam_...
1,80971,/data/Audi_dataset/20181204_135952/label3D/cam...,/data/Audi_dataset/20181204_135952/camera/cam_...
2,87534,/data/Audi_dataset/20181204_135952/label3D/cam...,/data/Audi_dataset/20181204_135952/camera/cam_...
3,113606,/data/Audi_dataset/20181204_135952/label3D/cam...,/data/Audi_dataset/20181204_135952/camera/cam_...
4,36780,/data/Audi_dataset/20181204_135952/label3D/cam...,/data/Audi_dataset/20181204_135952/camera/cam_...


In [5]:
train_dataset, test_dataset, val_dataset = np.split(dataset.sample(frac=1), 
                                                    [int(.7*len(dataset)), int(.8*len(dataset))])
train_dataset.shape, test_dataset.shape, val_dataset.shape

((8227, 3), (1176, 3), (2351, 3))

In [14]:
classesFile = './data/classes.txt'

def audi_convert(data):
    class_map = {name: idx for idx, name in enumerate(
        open(classesFile).read().splitlines())}
    logging.info("Class mapping loaded: %s", class_map)

    writer = tf.io.TFRecordWriter(output_file)

    for key, row in tqdm.tqdm(data.iterrows()):
        tf_example = build_example(row, class_map)
        writer.write(tf_example.SerializeToString())
    writer.close()
    logging.info("Done")

In [7]:
def build_example(row, class_map):
    d = pd.read_json(cur_dir + row['jsonFile'])
    d = d.loc[['2d_bbox','class']].transpose()    
    img_path = cur_dir + row['imgFile']
    height, width = plt.imread(img_path).shape[:2]
    img_raw = open(img_path, 'rb').read()
    key = hashlib.sha256(img_raw).hexdigest()

    xmin = []
    ymin = []
    xmax = []
    ymax = []
    classes = []
    classes_text = []
   
    for k, val in d.iterrows():
        x1 = val['2d_bbox'][0] if val['2d_bbox'][0] > 0 else 0
        y1 = val['2d_bbox'][1] if val['2d_bbox'][1] > 0 else 0
        x2 = val['2d_bbox'][2] if val['2d_bbox'][2] > 0 else 0
        y2 = val['2d_bbox'][3] if val['2d_bbox'][3] > 0 else 0
        
        xminval = (float(x1) / width) if (float(x1) / width) < 1 else 1
        yminval = (float(y1) / height) if (float(y1) / height) < 1 else 1
        xmaxval = (float(x2) / width) if (float(x2) / width) < 1 else 1
        ymaxval = (float(y2) / height) if (float(y2) / height) < 1 else 1
        
        xmin.append(xminval)
        ymin.append(yminval)
        xmax.append(xmaxval)
        ymax.append(ymaxval)
        classes_text.append(val['class'].encode('utf8'))
        classes.append(class_map[val['class']])

    example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[
            row['imgFile'].encode('utf8')])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[
            row['imgFile'].encode('utf8')])),
        'image/key/sha256': tf.train.Feature(bytes_list=tf.train.BytesList(value=[key.encode('utf8')])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=['png'.encode('utf8')])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmin)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmax)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymin)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymax)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
    }))

    return example

In [19]:
output_file = './data/audi_train.tfrecord'
audi_convert(train_dataset)

8227it [31:53,  4.35it/s]


In [15]:
output_file = './data/audi_val.tfrecord'
audi_convert(val_dataset)

2351it [04:29,  8.94it/s]
